In [63]:
import requests
import json
import datetime
import pandas as pd

In [64]:
USERNAME ='OOIAPI-9N9UMLHV9W5GOP'
TOKEN= 'SJN6HXHH116OZ8'

In [65]:
# define your inputs and starting time

refdes_streams = '/Users/knuth/Documents/ooi/repos/github/ooi_stats/botpt_streams.csv'
begin_time = datetime.date(2016, 7, 10)

In [66]:
# wrangling

# request only parameter 7 (time)
parameter = '7' 

# read in csv (for now)
refdes_streams_df = pd.read_csv(refdes_streams, parse_dates=True) # read in csv (for now).

# prepare time stamp related stuff
begin_time_str = begin_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
end_time = begin_time + datetime.timedelta(seconds=86400)
end_time_str = end_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
ntp_epoch = datetime.datetime(1900, 1, 1)
unix_epoch = datetime.datetime(1970, 1, 1)
ntp_delta = (unix_epoch - ntp_epoch).total_seconds()

# create some empty lists as inputs for your final data frame output
ref_des_list = []
stream_list = []
timestamp_list = []

In [73]:
%%time # use this to test performance

# interate over reference designators, delivery methods and streams

for index, row in refdes_streams_df.iterrows():
    ref_des =  row['ref_des']
    stream = row['stream']
    delivery_method = row['delivery_method']

    # print ref_des,stream,delivery_method
    
    # while begin_time < datetime.datetime.now().date(): # use this to run from begin time until present
    for i in range(5): # use for testing
        
        try:        
            response = requests.get('https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv/'+ref_des[:8]+'/'+ref_des[9:14]+'/'+ref_des[15:27]+'/'+delivery_method+'/'+stream+'?beginDT='+begin_time_str+'&endDT='+end_time_str+'&limit=1000&parameters='+parameter, auth=(USERNAME, TOKEN))
            data = response.json()
            timestamp = data[0]['time']
            timestamp = datetime.datetime.utcfromtimestamp(timestamp - ntp_delta).replace(microsecond=0)
            timestamp = timestamp.date()

            # print timestamp

            ref_des_list.append(ref_des)
            stream_list.append(stream)
            timestamp_list.append(timestamp)

        except:
            data = response.json()
            # print "\n From", begin_time_str, end_time_str, ":"
            # print data['status']


        # move to next day
        begin_time = begin_time + datetime.timedelta(seconds=86400)
        begin_time_str = begin_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')
        end_time = end_time + datetime.timedelta(seconds=86400)
        end_time_str = end_time.strftime('%Y-%m-%dT%H:%M:%S.000Z')

CPU times: user 2.45 s, sys: 111 ms, total: 2.56 s
Wall time: 1min 25s


In [74]:
# convert lists to data frame
data_dict = {
    'refdes':ref_des_list,
    'stream':stream_list,
    'timestamp':timestamp_list}
ooi_data = pd.DataFrame(data_dict, columns = ['refdes', 'stream', 'timestamp'])
print ooi_data

                          refdes             stream   timestamp
0    RS03CCAL-MJ03F-05-BOTPTA301  botpt_heat_sample  2016-07-09
1    RS03CCAL-MJ03F-05-BOTPTA301  botpt_heat_sample  2016-07-10
2    RS03CCAL-MJ03F-05-BOTPTA301  botpt_heat_sample  2016-07-12
3    RS03CCAL-MJ03F-05-BOTPTA301  botpt_heat_sample  2016-07-13
4    RS03CCAL-MJ03F-05-BOTPTA301  botpt_iris_sample  2016-07-15
5    RS03CCAL-MJ03F-05-BOTPTA301  botpt_iris_sample  2016-07-16
6    RS03CCAL-MJ03F-05-BOTPTA301  botpt_iris_sample  2016-07-17
7    RS03CCAL-MJ03F-05-BOTPTA301  botpt_iris_sample  2016-07-18
8    RS03CCAL-MJ03F-05-BOTPTA301  botpt_iris_sample  2016-07-19
9    RS03CCAL-MJ03F-05-BOTPTA301  botpt_lily_sample  2016-07-25
10   RS03CCAL-MJ03F-05-BOTPTA301  botpt_lily_sample  2016-07-26
11   RS03CCAL-MJ03F-05-BOTPTA301  botpt_lily_sample  2016-07-27
12   RS03CCAL-MJ03F-05-BOTPTA301  botpt_lily_sample  2016-07-28
13   RS03CCAL-MJ03F-05-BOTPTA301  botpt_lily_sample  2016-07-29
14   RS03CCAL-MJ03F-05-BOTPTA301  botpt_

In [75]:
# collapse by unique ref_des and day with data
test = ooi_data
test = test.drop('stream', axis = 1)
test = test.drop_duplicates()

In [76]:
ooi_data.head()

,refdes,stream,timestamp
0,RS03CCAL-MJ03F-05-BOTPTA301,botpt_heat_sample,2016-07-09
1,RS03CCAL-MJ03F-05-BOTPTA301,botpt_heat_sample,2016-07-10
2,RS03CCAL-MJ03F-05-BOTPTA301,botpt_heat_sample,2016-07-12
3,RS03CCAL-MJ03F-05-BOTPTA301,botpt_heat_sample,2016-07-13
4,RS03CCAL-MJ03F-05-BOTPTA301,botpt_iris_sample,2016-07-15


In [77]:
test.head()

,refdes,timestamp
0,RS03CCAL-MJ03F-05-BOTPTA301,2016-07-09
1,RS03CCAL-MJ03F-05-BOTPTA301,2016-07-10
2,RS03CCAL-MJ03F-05-BOTPTA301,2016-07-12
3,RS03CCAL-MJ03F-05-BOTPTA301,2016-07-13
4,RS03CCAL-MJ03F-05-BOTPTA301,2016-07-15
